In [1]:
from tqdm.notebook import tqdm

import torch # pytorch 라이브러리
import torch.nn as nn # 모델 구성을 위한 라이브러리
import torch.optim as optim # optimizer 설정을 위한 라이브러리
from torch.utils.data import Dataset, DataLoader # Dataset 설정을 위한 라이브러리
from torch.utils.tensorboard import SummaryWriter # TensorBoard 사용을 위한 클래스

import wandb # wandb 사용을 위한 라이브러리

import torchvision # pytorch의 컴퓨터 비전 라이브러리
import torchvision.transforms as T # 이미지 변환을 위한 모듈

In [2]:
# Seed 고정 함수 정의
import random
import torch.backends.cudnn as cudnn        # 그래픽 카드 사용용

def random_seed(seed_num):
    torch.manual_seed(seed_num)             # Pytorch의 CPU 난수 시드 고정
    torch.cuda.manual_seed(seed_num)        # Pytorch의 GPU 난수 시드 고정
    torch.cuda.manual_seed_all(seed_num)    # 다중 GPU 환경에서도 동일한 시드를 고정정
    cudnn.benchmark = False                 # 특정 연산 최적화 옵션을 매번 동일하게 유지
    cudnn.deterministic = True              # 같은 입력 데이터와 같은 시드를 사용했을 때 항상 동일한 출력 결과가 나오도록 보장
    random.seed(seed_num)                   # random 모듈의 시드 고정

random_seed(42)

In [3]:
mnist_transform = T.Compose([
    T.ToTensor()# Tensor 형식으로 변환
    ])

In [4]:
# torchvision 라이브러리에서 MNIST dataset 불러오기
download_root = './MNIST_DATASET'

train_dataset = torchvision.datasets.MNIST(download_root, transform=mnist_transform, train=True, download=True) # train dataset 다운로드
test_dataset = torchvision.datasets.MNIST(download_root, transform=mnist_transform, train=False, download=True) # train dataset 다운로드

In [5]:
# validation datasets 분리
total_size = len(train_dataset)
train_num, valid_num = int(total_size * 0.8), int(total_size * 0.2) # 20%만 val data로 빼냄
generator = torch.Generator().manual_seed(42)                       # pytorch의 난수 생성기 객체의 seed 값 고정정
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_num,valid_num], generator=generator)

### DataLoader
1. 주요 파라미터:
- dataset: 데이터를 로드할 데이터셋 객체 (예: TensorDataset, Dataset 상속 객체 등).
- batch_size: 한 번에 로드할 데이터의 개수(기본값: 1).
- shuffle: 데이터를 매 epoch마다 무작위로 섞을지 여부(기본값: False).
- num_workers: 데이터 로딩에 사용할 워커 프로세스 수(멀티스레딩).
- drop_last: 데이터 개수가 배치로 나누어 떨어지지 않을 경우 마지막 남은 데이터를 버릴지 여부.
2. 사용 이유:
- 효율적 미니배치 처리: 데이터를 일괄적으로 모델에 전달할 수 있습니다.
- 데이터 셔플링: 모델이 데이터 순서에 의존하지 않도록 합니다.
- 속도 개선: 멀티스레딩으로 데이터 로드 속도를 높일 수 있습니다.

In [6]:
# 앞서 선언한 Dataset을 인자로 주어 DataLoader를 선언
# 효율적 미니배치 처리 및 데이터 셔플링을 위해 사용
batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
class CNN(nn.Module):
    def __init__(self, num_classes, dropout_ratio):
        super(CNN,self).__init__()
        self.num_classes = num_classes

        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5), # [1,28,28]->[16,24,24]
            nn.ReLU(), # ReLU active function 적용
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5), # [16,24,24]->[32,20,20]
            nn.ReLU(), # ReLU active function 적용
            nn.MaxPool2d(kernel_size=2), # 크기를 1/2로 축소 [32,20,20]->[32,10,10]
            nn.Dropout(dropout_ratio),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5), # [32,10,10]->[64,6,6]
            nn.ReLU(), # ReLU active function 적용
            nn.MaxPool2d(kernel_size=2), # [64,6,6]->[64,3,3]
            nn.Dropout(dropout_ratio),
        )

        self.fc_layer = nn.Linear(64*3*3, self.num_classes) # [64,3,3] -> [num_classes]
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self,x):
        """
        input and output summary

        input:
            x: [batch_size,1,28,28]
        output:
            output: [batch_size,num_classes]
        """
        out = self.layer(x)             # [batch_size,64,3,3]
        out = out.view(x.shape[0], -1)  # [batch_size,576]
        pred = self.fc_layer(out)       # [batch_size,10]
        pred = self.softmax(pred)       # [batch_size,10]

        # 입력 데이터의 임베딩을 시각화 하기 위해 마지막 classifier layer를 통과하기 전 출력값(out)을 기록
        return pred, out

    def weight_initialization(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)


In [8]:
# TensorBoard는 로그 파일을 이용해 시각화가 가능하다. "SummaryWriter" 사용
# 인자로 로그 파일이 저장될 위치를 지정합니다. 로그 데이터는 '.event'파일로 저장됨.

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("./runs/tutorial")

### add_scalar
1. 주요 인자
- tag: 시각화할 데이터의 이름(예: 'Loss/train', 'Accuracy/validation' 등).
- scalar_value: 기록할 스칼라 값(예: 손실, 정확도 등).
- global_step: 현재 단계 또는 에포크를 나타내는 값. 이 값에 따라 시각화할 그래프의 x축이 설정됩니다.
2. 역할
- add_scalar를 통해 학습 중 손실 값, 정확도, 학습률 등을 기록하고 TensorBoard에서 쉽게 그래프로 확인할 수 있습니다.
- 여러 값을 add_scalar로 기록하여 학습 과정 전반의 동향을 모니터링할 수 있습니다.

In [9]:
# add_scalar
writer.add_scalar(tag="Metric/valid_accuracy", scalar_value=0.80, global_step=1)
writer.add_scalar(tag="Metric/valid_accuracy", scalar_value=0.85, global_step=2)
writer.add_scalar(tag="Metric/valid_accuracy", scalar_value=0.90, global_step=3)
writer.add_scalar(tag="Metric/valid_accuracy", scalar_value=0.95, global_step=4)

### Tensorboard 실행
터미널에서 실행하는 경우 pip install tensorboard를 통해 설치후 터미널에서 실행  
tensorboard --logdir=[log-directory-path]  
해당 명령어를 통해 tensorboard 실행 가능 이때 log-directory-path는 수정해 줘야 됨.  
해당 파일에서는  
tensorboard --logdir=C:/Users/KJH/Code_Document/TIL/python_note/runs/tutorial

IPython의 경우  
%load_ext tensorboard  
를 통해 IPython 환경에서 확장 프로그램인 tensorboard를 사용할 수 있도록 하는 매직 커맨드를 작성해 주고  
%tensorboard --logdir=[log-directory-path]  
해당 명령어를 통해   
%tensorboard --logdir=C:/Users/KJH/Code_Document/TIL/python_note/runs/tutorial  


In [10]:
# add graph
dropout_ratio = 0.2
model = CNN(num_classes = 10, dropout_ratio = dropout_ratio)
model.weight_initialization()

for images, labels in train_dataloader:
    break

# 모델의 구조를 로깅합니다. 'graphs 탭을 클릭해 확인할 수 있습니다.
# add_graph에서 input_to_model 을 인자로 넣어, 모델을 거치면서 tensor의 차원 변화를 알 수 있습니다.
writer.add_graph(model=model, input_to_model=images)

In [11]:
# add_histogram
# weight와 bias에 대해 histogram을 epoch 별로 표현해 줌
for name, weight in model.named_parameters():
    writer.add_histogram(tag=name,values=weight)

In [12]:
img_grid = torchvision.utils.make_grid(images)
writer.add_image('MNIST Images', img_grid)

In [13]:
# add_embedding
# image data를 emvedding이라 가정하고 시각화
# 고차원 vector를 로깅함. "projector"탭을 클릭해 고차원 벡터를 2d,3d 공간에 정사영한 결과물을 호가인 가능
for images, labels in train_dataloader:
    break
embedding = images.view(images.size(0), -1)
writer.add_embedding(mat = embedding, metadata=labels, label_img=images, global_step=1)

In [14]:
def training(model, dataloader, train_dataset, criterion, optimizer, device, epoch, num_epochs):
  model.train()  # 모델을 학습 모드로 설정
  train_loss = 0.0
  train_accuracy = 0

  tbar = tqdm(dataloader)
  for images, labels in tbar:
      images = images.to(device)
      labels = labels.to(device)

      # 순전파
      outputs, _ = model(images)
      loss = criterion(outputs, labels)

      # 역전파 및 가중치 업데이트
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # 손실과 정확도 계산
      train_loss += loss.item()
      # torch.max에서 dim 인자에 값을 추가할 경우, 해당 dimension에서 최댓값과 최댓값에 해당하는 인덱스를 반환
      _, predicted = torch.max(outputs, 1)
      train_accuracy += (predicted == labels).sum().item()

      # tqdm의 진행바에 표시될 설명 텍스트를 설정
      tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}")

  # 에폭별 학습 결과 출력
  train_loss = train_loss / len(dataloader)
  train_accuracy = train_accuracy / len(train_dataset)

  return model, train_loss, train_accuracy

def evaluation(model, dataloader, valid_dataset, criterion, device, epoch, num_epochs):
  model.eval()  # 모델을 평가 모드로 설정
  valid_loss = 0.0
  valid_accuracy = 0

  with torch.no_grad(): # model의 업데이트 막기
      tbar = tqdm(dataloader)
      for images, labels in tbar:
          images = images.to(device)
          labels = labels.to(device)

          # 순전파
          outputs, _ = model(images)
          loss = criterion(outputs, labels)

          # 손실과 정확도 계산
          valid_loss += loss.item()
          # torch.max에서 dim 인자에 값을 추가할 경우, 해당 dimension에서 최댓값과 최댓값에 해당하는 인덱스를 반환
          _, predicted = torch.max(outputs, 1)
          valid_accuracy += (predicted == labels).sum().item()

          # tqdm의 진행바에 표시될 설명 텍스트를 설정
          tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Valid Loss: {loss.item():.4f}")

  valid_loss = valid_loss / len(dataloader)
  valid_accuracy = valid_accuracy / len(valid_dataset)

  return model, valid_loss, valid_accuracy


def training_loop(model, train_dataloader, valid_dataloader, criterion, optimizer, device, num_epochs, patience, model_name, writer):
    best_valid_loss = float('inf')  # 가장 좋은 validation loss를 저장
    early_stop_counter = 0  # 카운터
    valid_max_accuracy = -1

    for epoch in range(num_epochs):
        model, train_loss, train_accuracy = training(model, train_dataloader, train_dataset, criterion, optimizer, device, epoch, num_epochs)
        model, valid_loss, valid_accuracy = evaluation(model, valid_dataloader, valid_dataset, criterion, device, epoch, num_epochs)

        # loss, accuracy 로깅
        writer.add_scalar("Loss/train_loss", train_loss, epoch)
        writer.add_scalar("Loss/valid_loss", valid_loss, epoch)
        writer.add_scalar("Metric/train_accuracy", train_accuracy, epoch)
        writer.add_scalar("Metric/valid_accuracy", valid_accuracy, epoch)

        if valid_accuracy > valid_max_accuracy:
          valid_max_accuracy = valid_accuracy

        # validation loss가 감소하면 모델 저장 및 카운터 리셋
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f"./model_{model_name}.pt")
            early_stop_counter = 0

        # validation loss가 증가하거나 같으면 카운터 증가
        else:
            early_stop_counter += 1

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f} Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

        # 조기 종료 카운터가 설정한 patience를 초과하면 학습 종료
        if early_stop_counter >= patience:
            print("Early stopping")
            break

    return model, valid_max_accuracy

In [15]:
writer = SummaryWriter("./runs/training")
lr = 0.001
dropout_ratio = 0.2
num_epochs = 100
patience = 3
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # gpu 설정
model_name = 'exp1'
num_classes = 10

model = CNN(num_classes = 10, dropout_ratio = dropout_ratio)
model.weight_initialization()
model = model.to(device)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)
model, valid_max_accuracy = training_loop(model, train_dataloader, valid_dataloader, criterion, optimizer, device, num_epochs, patience, model_name, writer)

  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

Epoch [1/100], Train Loss: 0.1646, Train Accuracy: 0.9486 Valid Loss: 0.0581, Valid Accuracy: 0.9821


  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

...

  0%|          | 0/375 [00:00<?, ?it/s]

Epoch [7/100], Train Loss: 0.0225, Train Accuracy: 0.9927 Valid Loss: 0.0332, Valid Accuracy: 0.9907


  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

Epoch [8/100], Train Loss: 0.0190, Train Accuracy: 0.9940 Valid Loss: 0.0330, Valid Accuracy: 0.9918
Early stopping


In [16]:
# test 이미지 데이터를 가져와 모델로부터 계산된 임베딩을 TensorBoard로 시각화합니다.
# test 이미지 데이터를 가져옵니다.

model.eval()  # Evaluation 모드
with torch.no_grad():
  for images,labels in test_dataloader:
      _, embeddings = model(images.to(device))  # self.fc2의 출력을 가져옵니다.
      break # test data가 10,000개므로 모두 시각화를 진행한다면, 결과를 한 눈에 파악하기 어려울 수 있습니다. 실습에서는 간단히 하나의 미니 배치에 대해서만 실습을 진행합니다.

# 임베딩 결과를 CPU로 가져와서 Numpy 형태로 변환합니다.
embeddings = embeddings.cpu().numpy()

# 텐서보드에 임베딩을 기록합니다.
writer.add_embedding(
    mat=embeddings,
    metadata=labels,  # 레이블을 metadata로 설정하여 각 이미지가 어떤 클래스에 속하는지 알 수 있게 합니다.
    label_img=images,  # 이미지도 함께 보여주기 위해 설정합니다.
    global_step=1,
)

In [18]:
# 기존까지는 tensorboard에서 log가 저장되는 위치를 구분하여 로드하였습니다. (e.g. ./runs/tutorial, ./runs/training)
# 로그 파일이 저장되는 폴더를 포괄하는 폴더(e.g. 부모 디렉토리)를 --logdir로 설정한다면, log를 overlap하여 시각화하고 볼 수 있습니다.
# valid_accuracy에서 overlap을 확인할 수 있습니다.
%reload_ext tensorboard
%tensorboard --logdir ./runs --port 6008

Reusing TensorBoard on port 6008 (pid 29016), started 0:00:39 ago. (Use '!kill 29016' to kill it.)

In [20]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\KJH\_netrc
wandb: Currently logged in as: jihu6033 (slowin-fc_comapny) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [21]:
# wandb.init을 통해 프로젝트와 실험을 생성합니다.
run = wandb.init(project = 'test-mnist', name = 'tutorial')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [22]:
# 간단하게 매 epoch마다 accuracy를 기록했다고 생각하고, 로깅합니다.
run.log({"accuracy": 0.8}, step = 1)  # wandb.log
run.log({"accuracy": 0.85}, step = 2)
run.log({"accuracy": 0.9}, step = 3)
run.log({"accuracy": 0.95}, step = 4)

In [24]:
# 모델의 예측 결과와 라벨을 통해 wandb에서 confusion matrix를 시각화합니다.
total_preds = [1, 0, 0, 0, 1, 1, 1, 1, 0, 0]
total_labels = [1, 0, 0, 0, 0, 1, 1, 1, 0, 0]

wandb.sklearn.plot_confusion_matrix(total_labels, total_preds, labels=['0', '1'])

In [25]:
# WandB에 이미지를 로깅합니다.
for images, labels in train_dataloader:
  break

run.log({'mnist-example': [wandb.Image(image, caption = str(label)) for image, label in zip(images, labels)]})

In [26]:
run.finish()

accuracy,▁▃▆█
accuracy,0.95


In [27]:
def training(model, dataloader, train_dataset, criterion, optimizer, device, epoch, num_epochs):
  model.train()  # 모델을 학습 모드로 설정
  train_loss = 0.0
  train_accuracy = 0

  tbar = tqdm(dataloader)
  for images, labels in tbar:
      images = images.to(device)
      labels = labels.to(device)

      # 순전파
      outputs, _ = model(images)
      loss = criterion(outputs, labels)

      # 역전파 및 가중치 업데이트
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # 손실과 정확도 계산
      train_loss += loss.item()
      # torch.max에서 dim 인자에 값을 추가할 경우, 해당 dimension에서 최댓값과 최댓값에 해당하는 인덱스를 반환
      _, predicted = torch.max(outputs, 1)
      train_accuracy += (predicted == labels).sum().item()

      # tqdm의 진행바에 표시될 설명 텍스트를 설정
      tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}")

  # 에폭별 학습 결과 출력
  train_loss = train_loss / len(dataloader)
  train_accuracy = train_accuracy / len(train_dataset)

  return model, train_loss, train_accuracy

def evaluation(model, dataloader, valid_dataset, criterion, device, epoch, num_epochs):
  model.eval()  # 모델을 평가 모드로 설정
  valid_loss = 0.0
  valid_accuracy = 0

  with torch.no_grad(): # model의 업데이트 막기
      tbar = tqdm(dataloader)
      for images, labels in tbar:
          images = images.to(device)
          labels = labels.to(device)

          # 순전파
          outputs, _ = model(images)
          loss = criterion(outputs, labels)

          # 손실과 정확도 계산
          valid_loss += loss.item()
          # torch.max에서 dim 인자에 값을 추가할 경우, 해당 dimension에서 최댓값과 최댓값에 해당하는 인덱스를 반환
          _, predicted = torch.max(outputs, 1)
          valid_accuracy += (predicted == labels).sum().item()

          # tqdm의 진행바에 표시될 설명 텍스트를 설정
          tbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Valid Loss: {loss.item():.4f}")

  valid_loss = valid_loss / len(dataloader)
  valid_accuracy = valid_accuracy / len(valid_dataset)

  return model, valid_loss, valid_accuracy

def training_loop(model, train_dataloader, valid_dataloader, criterion, optimizer, device, num_epochs, patience, model_name, run):
    best_valid_loss = float('inf')  # 가장 좋은 validation loss를 저장
    early_stop_counter = 0  # 카운터
    valid_max_accuracy = -1

    for epoch in range(num_epochs):
        model, train_loss, train_accuracy = training(model, train_dataloader, train_dataset, criterion, optimizer, device, epoch, num_epochs)
        model, valid_loss, valid_accuracy = evaluation(model, valid_dataloader, valid_dataset, criterion, device, epoch, num_epochs)

        monitoring_value = {'train_loss': train_loss, 'train_accuracy': train_accuracy, 'valid_loss': valid_loss, 'valid_accuracy': valid_accuracy}
        run.log(monitoring_value, step=epoch)               # 해당 부근에서 log를 저장

        if valid_accuracy > valid_max_accuracy:
          valid_max_accuracy = valid_accuracy

        # validation loss가 감소하면 모델 저장 및 카운터 리셋
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f"./model_{model_name}.pt")
            early_stop_counter = 0

        # validation loss가 증가하거나 같으면 카운터 증가
        else:
            early_stop_counter += 1

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f} Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

        # 조기 종료 카운터가 설정한 patience를 초과하면 학습 종료
        if early_stop_counter >= patience:
            print("Early stopping")
            break

    return model, valid_max_accuracy

In [28]:
run = wandb.init(project = 'test-mnist', name = 'training')

lr = 0.001
dropout_ratio = 0.2
num_epochs = 100
patience = 3
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # gpu 설정
model_name = 'exp1'
num_classes = 10

model = CNN(num_classes = 10, dropout_ratio = dropout_ratio)
model.weight_initialization()
model = model.to(device)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

# wandb에 모델의 weight & bias, graident를 시각화합니다.
run.watch(model, criterion, log = 'all', log_graph = True)

model, valid_max_accuracy = training_loop(model, train_dataloader, valid_dataloader, criterion, optimizer, device, num_epochs, patience, model_name, run)
run.finish()

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

Epoch [1/100], Train Loss: 0.1733, Train Accuracy: 0.9455 Valid Loss: 0.0599, Valid Accuracy: 0.9812


  0%|          | 0/1500 [00:00<?, ?it/s]

...

Epoch [9/100], Train Loss: 0.0162, Train Accuracy: 0.9945 Valid Loss: 0.0376, Valid Accuracy: 0.9902


  0%|          | 0/1500 [00:00<?, ?it/s]

  0%|          | 0/375 [00:00<?, ?it/s]

Epoch [10/100], Train Loss: 0.0176, Train Accuracy: 0.9945 Valid Loss: 0.0358, Valid Accuracy: 0.9908
Early stopping


train_accuracy,▁▆▇▇▇█████
train_loss,█▃▂▂▂▁▁▁▁▁
valid_accuracy,▁▂▅▆████▇█
valid_loss,█▇▄▃▁▁▁▁▃▂
train_accuracy,0.99454
train_loss,0.0176
valid_accuracy,0.99075
valid_loss,0.0358
